[View in Colaboratory](https://colab.research.google.com/github/hajmolafury/learning-compression/blob/master/mnist_updated.ipynb)

In [0]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL']='2'

In [0]:
import time
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import numpy as np

from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly
plotly.tools.set_credentials_file(username='ymehta95', api_key='TVHPG5E24omQeUyWougP')
import plotly.plotly as py
import plotly.graph_objs as go

#To install plotpy library:
#pip install plotly --upgrade

In [107]:
mnist = input_data.read_data_sets("MNIST_data/",one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [0]:
nodes_hl1=1000
nodes_hl2=1000
n_classes=10
batch_size=128

In [0]:
x=tf.placeholder('float',[None,784])
y=tf.placeholder('float')
keep_prob=tf.placeholder(tf.float32)

In [0]:
def neural_network(data):
    hl1={'weights':tf.Variable(tf.random_normal([784,nodes_hl1])),'bias':tf.Variable(tf.random_normal([nodes_hl1]))}
    hl2={'weights':tf.Variable(tf.random_normal([nodes_hl1,nodes_hl2])),'bias':tf.Variable(tf.random_normal([nodes_hl2]))}
    output_layer={'weights':tf.Variable(tf.random_normal([nodes_hl2,n_classes])),'bias':tf.Variable(tf.random_normal([n_classes]))}
    
    #Activation function : ReLU
    
    l1=tf.nn.relu(tf.add(tf.matmul(data,hl1['weights']),hl1['bias']))
    l1_dropout=tf.nn.dropout(l1,keep_prob)
                                                                                        
    l2=tf.nn.relu(tf.add(tf.matmul(l1,hl2['weights']),hl2['bias']))
    #l2_dropout=tf.nn.dropout(l2,keep_prob)
                                                                                           
    output=tf.add(tf.matmul(l2,output_layer['weights']),output_layer['bias'])
                                                                                           
    return output

In [111]:
summary_dir = '/tmp/tensorflow/mnist'
mnist.validation.num_examples

5000

In [0]:
test_acc_adam=[]
test_acc_rmsprop=[]
test_acc_sgd=[]

In [0]:
def train_neural_network(x,optimizer):
    prediction=neural_network(x)
    cost=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=prediction,labels=y))
    if optimizer=='adam':
        optimize=tf.train.AdamOptimizer(learning_rate=0.003).minimize(cost)
    if optimizer=='sgd':
        optimize=tf.train.GradientDescentOptimizer(learning_rate=0.001).minimize(cost)
    if optimizer=='rmsprop':
        optimize=tf.train.RMSPropOptimizer(learning_rate=0.001,decay=0.9,momentum=0.9).minimize(cost)
        
   
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        
        for epoch in range(num_epoch):
           
            for i in range(int(mnist.train.num_examples/batch_size)):
                batch_x,batch_y=mnist.train.next_batch(batch_size)
                _, c=sess.run([optimize, cost],feed_dict={x:batch_x,y:batch_y,keep_prob:0.8})
                epoch_loss[epoch]+=c
            
                if i%int((int(mnist.train.num_examples/batch_size)/f))==0:
                    correct=tf.equal(tf.argmax(prediction,1),tf.argmax(y,1))
                    accuracy=tf.reduce_mean(tf.cast(correct,'float'))
                    val_acc.append(accuracy.eval({x:mnist.validation.images,y:mnist.validation.labels,keep_prob:1}))
                    train_acc.append(accuracy.eval({x:mnist.train.images,y:mnist.train.labels,keep_prob:1}))
                    test_acc.append(accuracy.eval({x:mnist.test.images,y:mnist.test.labels,keep_prob:1}))
            
            print ('Epoch', epoch+1,'completed out of',num_epoch,'val acc: ', val_acc[-1])
            #print ('train acc: ',train_acc[-1])
        print (time.time() - start_time, 'sec\n')
        print('\nTEST ACC : ',accuracy.eval({x:mnist.test.images,y:mnist.test.labels,keep_prob:1}),'\n')
        
    

In [114]:
num_epoch=20
f=2 #frequency : number of times in an epoch we want to measure the accuracy
epoch_loss=np.zeros(num_epoch)
val_acc=[]
train_acc=[]
test_acc=[]

if __name__=="__main__":
    start_time=time.time()
    train_neural_network(x,'adam')
    print ("Training Over. \nNow you are a SuperSaiyan\n")  
    test_acc_adam=test_acc

Epoch 1 completed out of 20 val acc:  0.9452
Epoch 2 completed out of 20 val acc:  0.9542
Epoch 3 completed out of 20 val acc:  0.9592
Epoch 4 completed out of 20 val acc:  0.9636
Epoch 5 completed out of 20 val acc:  0.9678
Epoch 6 completed out of 20 val acc:  0.9654
Epoch 7 completed out of 20 val acc:  0.9674
Epoch 8 completed out of 20 val acc:  0.9714
Epoch 9 completed out of 20 val acc:  0.9714
Epoch 10 completed out of 20 val acc:  0.9708
Epoch 11 completed out of 20 val acc:  0.971
Epoch 12 completed out of 20 val acc:  0.9668
Epoch 13 completed out of 20 val acc:  0.9734
Epoch 14 completed out of 20 val acc:  0.971
Epoch 15 completed out of 20 val acc:  0.9742
Epoch 16 completed out of 20 val acc:  0.9764
Epoch 17 completed out of 20 val acc:  0.973
Epoch 18 completed out of 20 val acc:  0.9784
Epoch 19 completed out of 20 val acc:  0.9802
Epoch 20 completed out of 20 val acc:  0.9726
126.46504473686218 sec


TEST ACC :  0.9704 

Training Over. 
Now you are a SuperSaiyan



In [0]:
x_axis1=np.arange(num_epoch)
x_axis2= np.arange(0,num_epoch,1/f)

In [116]:
loss_go=go.Scatter(x=x_axis1,y=epoch_loss)
val_acc_go=go.Scatter(x=x_axis2[1:],y=val_acc[1:], name='validation accuracy')
train_acc_go=go.Scatter(x=x_axis2[1:],y=train_acc[1:], name='training accuracy')
test_acc_go=go.Scatter(x=x_axis2[1:],y=test_acc[1:], name='testing accuracy')

data=([loss_go])
py.iplot(data,filename='adam_loss')

In [117]:
data=([val_acc_go,train_acc_go,test_acc_go])
py.iplot(data,filename='adam_acc')

In [118]:
#training with RMSprop
num_epoch=20
f=2
epoch_loss=np.zeros(num_epoch)
val_acc=[]
train_acc=[]
test_acc=[]

if __name__=="__main__":
    start_time=time.time()
    train_neural_network(x,'rmsprop')
    print ("Training Over. \nNow you are a SuperSaiyan\n")
    test_acc_rmsprop=test_acc

Epoch 1 completed out of 20 val acc:  0.9576
Epoch 2 completed out of 20 val acc:  0.9582
Epoch 3 completed out of 20 val acc:  0.9592
Epoch 4 completed out of 20 val acc:  0.9678
Epoch 5 completed out of 20 val acc:  0.9642
Epoch 6 completed out of 20 val acc:  0.9632
Epoch 7 completed out of 20 val acc:  0.9726
Epoch 8 completed out of 20 val acc:  0.9728
Epoch 9 completed out of 20 val acc:  0.9706
Epoch 10 completed out of 20 val acc:  0.9738
Epoch 11 completed out of 20 val acc:  0.9762
Epoch 12 completed out of 20 val acc:  0.9738
Epoch 13 completed out of 20 val acc:  0.9748
Epoch 14 completed out of 20 val acc:  0.9714
Epoch 15 completed out of 20 val acc:  0.9774
Epoch 16 completed out of 20 val acc:  0.977
Epoch 17 completed out of 20 val acc:  0.9746
Epoch 18 completed out of 20 val acc:  0.976
Epoch 19 completed out of 20 val acc:  0.9738
Epoch 20 completed out of 20 val acc:  0.9792
128.02450919151306 sec


TEST ACC :  0.9766 

Training Over. 
Now you are a SuperSaiyan



In [119]:
loss_go=go.Scatter(x=x_axis1,y=epoch_loss)
val_acc_go=go.Scatter(x=x_axis2[1:],y=val_acc[1:], name='validation accuracy')
train_acc_go=go.Scatter(x=x_axis2[1:],y=train_acc[1:], name='training accuracy')
test_acc_go=go.Scatter(x=x_axis2[1:],y=test_acc[1:], name='testing accuracy')

data=([loss_go])
py.iplot(data,filename='rms_loss')

In [120]:
data=([val_acc_go,train_acc_go,test_acc_go])
py.iplot(data,filename='rms_acc')

In [121]:
#training with SGD
num_epoch=20
f=2
epoch_loss=np.zeros(num_epoch)
val_acc=[]
train_acc=[]
test_acc=[]

if __name__=="__main__":
    start_time=time.time()
    train_neural_network(x,'sgd')
    print ("Training Over. \nNow you are a SuperSaiyan\n")
    test_acc_sgd=test_acc

Epoch 1 completed out of 20 val acc:  0.8874
Epoch 2 completed out of 20 val acc:  0.9028
Epoch 3 completed out of 20 val acc:  0.9136
Epoch 4 completed out of 20 val acc:  0.92
Epoch 5 completed out of 20 val acc:  0.9236
Epoch 6 completed out of 20 val acc:  0.9286
Epoch 7 completed out of 20 val acc:  0.9282
Epoch 8 completed out of 20 val acc:  0.9308
Epoch 9 completed out of 20 val acc:  0.9346
Epoch 10 completed out of 20 val acc:  0.9338
Epoch 11 completed out of 20 val acc:  0.936
Epoch 12 completed out of 20 val acc:  0.9358
Epoch 13 completed out of 20 val acc:  0.9346
Epoch 14 completed out of 20 val acc:  0.939
Epoch 15 completed out of 20 val acc:  0.9362
Epoch 16 completed out of 20 val acc:  0.9376
Epoch 17 completed out of 20 val acc:  0.9352
Epoch 18 completed out of 20 val acc:  0.9388
Epoch 19 completed out of 20 val acc:  0.9384
Epoch 20 completed out of 20 val acc:  0.9374
126.8205235004425 sec


TEST ACC :  0.9381 

Training Over. 
Now you are a SuperSaiyan



In [122]:
loss_go=go.Scatter(x=x_axis1,y=epoch_loss)
val_acc_go=go.Scatter(x=x_axis2[1:],y=val_acc[1:], name='validation accuracy')
train_acc_go=go.Scatter(x=x_axis2[1:],y=train_acc[1:], name='training accuracy')
test_acc_go=go.Scatter(x=x_axis2[1:],y=test_acc[1:], name='testing accuracy')

data=([loss_go])
py.iplot(data,filename='sgd_loss')

In [123]:
data=([val_acc_go,train_acc_go,test_acc_go])
py.iplot(data,filename='sgd_acc')

In [124]:
test_acc_adam_go=go.Scatter(x=x_axis2[1:],y=test_acc_adam[1:], name='Adam')
test_acc_rmsprop_go=go.Scatter(x=x_axis2[1:],y=test_acc_rmsprop[1:], name='RMSProp')
test_acc_sgd_go=go.Scatter(x=x_axis2[1:],y=test_acc_sgd[1:], name='SGD')

data=([test_acc_adam_go,test_acc_rmsprop_go,test_acc_sgd_go])
py.iplot(data,filename='accuracy comparison')